### JG: use Selenium to check Tesco delivery availability for the next 3 weeks.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time 
from datetime import date, timedelta

In [2]:
# headless Chrome setup
options = Options()
options.headless = True
DRIVER = webdriver.Chrome(options=options)

In [3]:
# Read the username and password from a file
for line in open("login_details.txt","r").readlines():
    loginInfo = line.split(":")
    USERNAME = loginInfo[0]
    PASS = loginInfo[1]

In [4]:
def account_login(driver,username,password):  
    driver.get("https://secure.tesco.com/account/en-GB/login?from=/")
   
    # input username
    username_box = driver.find_element_by_id('username')
    username_box.click()
    username_box.send_keys(f"{username}")
    
    # input password
    password_box = driver.find_element_by_id('password')
    password_box.click()
    password_box.send_keys(f'{password}')
    
    # login
    tag_list = driver.find_elements_by_xpath('//*[@id="sign-in-form"]/button/span')
    tag_list[0].click()


def check_delivery(driver,date_to_check):
    driver.get(f"https://www.tesco.com/groceries/en-GB/slots/delivery/{date_to_check}?slotGroup=4")
    
    # load the page content
    time.sleep(3) 
    element = driver.find_element_by_id('content')
    website = element.text
    na_message="No slots available! Try another day"
    
    if na_message in website:
        return 0
    else:
        return 1
    
def availability_report(driver):
    today = date.today()
    
    # check next 3 weeks
    dates_to_check= [today,today +timedelta(days=7),
                     today + timedelta(days=14)]
    # result to dict
    availability_report={}
    for d in dates_to_check:
        availability_report[str(d)]=check_delivery(driver,d)
        
    return availability_report

In [5]:
# login & scrape the website
account_login(driver=DRIVER,username=USERNAME,password=PASS)
report=availability_report(driver=DRIVER)

In [6]:
print('Delivery available in weeks commencing:\n')
for k, v in report.items():
    print(k, ': ', v)

Delivery available in weeks commencing:

2020-04-16 :  0
2020-04-23 :  0
2020-04-30 :  0


In [7]:
if sum(report.values())<1:
    print("No deliveries available in next 3 weeks...")
else:
    print("Deliveries available!!!")

No deliveries available in next 3 weeks...


In [8]:
DRIVER.quit()